# Blight Data Analysis

This analysis is being performed to predict whether or not a person will be in compliance with a blight ticket.

### Load the Libraries

In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

### Import the Datasets

In [27]:
data_load = pd.read_csv('https://data.detroitmi.gov/api/views/ti6p-wcg4/rows.csv?accessType=DOWNLOAD&api_foundry=true', engine='python', index_col='Ticket ID')

In [25]:
data_load.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 408330 entries, 445536 to 190940
Data columns (total 39 columns):
Ticket Number                           408330 non-null object
Agency Name                             408330 non-null object
Inspector Name                          408330 non-null object
Violator Name                           408328 non-null object
Violator ID                             408330 non-null int64
Violation Street Number                 408330 non-null int64
Violation Street Name                   408265 non-null object
Violation Zip Code                      76814 non-null object
Mailing Address Street Number           408326 non-null object
Mailing Address Street Name             408324 non-null object
Mailing Address City                    406069 non-null object
Mailing Address State                   405607 non-null object
Mailing Address Zip Code                406067 non-null object
Mailing Address Non-USA Code            2263 non-null object
Mailing